# 0L Contributors payments report 
## Generate data
### Pre-requisites
- access to a well formed contributors.json file
- a correctly configured config.py file
### TODO
- add a check on sender address

### Imports and loading variables

In [1]:
import requests
import json
import pandas as pd
import csv
import os

from config import Config

f = open(f"{Config.PROJECT_PATH}/assets/contributors.json")
contributors = json.load(f)
print(contributors)

[{'account': '5F8AC83A9B3BF2EFF20A6C16CD05C111', 'identities': [{'context': 'Discord', 'id': '776520842070720514', 'username': 'Paulito#0564'}, {'context': 'Github', 'id': 'paulito123', 'username': 'Paulito'}], 'roles': [{'name': 'broker', 'context': 'analytics-wg'}]}, {'account': '5DB97DD068263449C8EB369DC2312AFA', 'identities': [{'context': 'Discord', 'id': '776520842070799999', 'username': 'Paulito#9999'}, {'context': 'Github', 'id': 'paulito999', 'username': 'Dummito'}], 'roles': [{'name': 'developer', 'context': 'analytics-wg'}]}, {'account': 'E8DF1AFD258A10BD0392E1A8D274CC81', 'identities': [{'context': 'Discord', 'id': '776520842070700000', 'username': 'Paulito#0000'}, {'context': 'Github', 'id': 'paulito000', 'username': 'Dummito2'}], 'roles': [{'name': 'developer', 'context': 'hustle-karma-wg'}]}]


### Get payments and generate the payment.json file
Here we will use an account list, generated from the contributors.json file, to iterate the accounts and fetch transactions which are relevant for us. On the fly, we are fetching on-chain transaction data that can be linked to real world data and generate a payments.json file which will have place holders for custom information linked to the payments.

In [2]:
# extract all addresses to be evaluated
address_list = [contributor['account'] for contributor in contributors]

# result list of all transactions, wrapped by mission data
result = []

# get all payments for address list between the given block height
for addr in address_list:
    
    end_not_reached = True
    start = 0
    limit = 1000
    while end_not_reached:
        url = f"https://0lexplorer.io/api/proxy/node/events?address={addr}&start={start}&limit={limit}"
        response = requests.get(url)
        response_json = response.json()
        if "result" in response_json:
            payload = response_json["result"]
            # check the payload for data of interest
            for tx in reversed(payload):
                if tx['transaction_version'] >= Config.START_BLOCK:
                    if tx['transaction_version'] <= Config.END_BLOCK and \
                       tx['data']['type'] in ['receivedpayment']:
                        # create payment entry
                        payment = {
                            "contributor": f"{addr}",
                            "workgroup_id": "MANUAL INPUT",     # analytics-wg, ... 
                            "assignment_ref": "MANUAL INPUT",   # URL to airtable of custom ID?
                            "payment_type": "MANUAL INPUT",     # work-related, bonus, rectification, ...
                            "description": "MANUAL INPUT",      # Short description for reporting
                            "contributor_role": "MANUAL INPUT", # broker, developer, analyst, ...
                            "reviewer": "MANUAL INPUT",         # broker role?
                            "tx": tx,
                            "artifacts":["MANUAL INPUT"],       # Links to work delivered if applicable
                            }
                        result.append(payment)
                else:
                    break

            if len(payload) < 1000:
                end_not_reached = False
            else:
                start += 1000
                limit += 1000
        else:
            end_not_reached = False
    
print(result)
        

[]


### write payments data to json file 

In [3]:
fq_filename = f"{Config.PROJECT_PATH}/assets/generated/payments/payments_{Config.PAYMENT_ROUND}.json"

# write result pretty-printed to a file
payment_file = open(fq_filename, "w")
payment_file.write(json.dumps(result, indent=4, sort_keys=True))
payment_file.close()

print(f"{fq_filename} created...")

/home/user/projects/ol-analytical-research/assets/generated/payments/payments_PR20221001_002.json created...


## Generate a report
### Merge data files

In [4]:
# assign directory
directory = f'{Config.PROJECT_PATH}/assets/generated/payments'

# define output
output = []

print("loading files...")
 
# iterate payment files
for filename in os.listdir(directory):
    fq_filename = os.path.join(directory, filename)

    if os.path.isfile(fq_filename) and filename[-5:] == ".json":
        print(fq_filename)
        with open(fq_filename, 'r') as f:
            data = json.load(f)
            output += data

print("")
print("OUTPUT")
print(output)

loading files...
/home/user/projects/ol-analytical-research/assets/generated/payments/payments_PR20221001_001.json
/home/user/projects/ol-analytical-research/assets/generated/payments/payments_PR20221010_001.json
/home/user/projects/ol-analytical-research/assets/generated/payments/payments_PR20221001_002.json

OUTPUT
[{'artifacts': ['https://github.com/paulito123/ol-analytical-research'], 'contributor': '5F8AC83A9B3BF2EFF20A6C16CD05C111', 'contributor_role': 'broker', 'description': 'Coordinating the project, developing 50%', 'workgroup_id': 'analytics-wg', 'assignment_ref': 'https://airtable.com/appuiDshjiBzOAGl2/tblixhNzkEaKt3a75/viw77ncDURPkx2Mln/rec4D7I4k2KS4ytt5?blocks=hide', 'payment_type': 'work-related', 'reviewer': '5F8AC83A9B3BF2EFF20A6C16CD05C111', 'tx': {'data': {'amount': {'amount': 500000000, 'currency': 'GAS'}, 'metadata': '6163636f756e742067656e65726174696f6e', 'receiver': '5f8ac83a9b3bf2eff20a6c16cd05c111', 'sender': '0cb4fef910b686e5ae39049fb892cb24', 'type': 'receive

### Reorganise, transform and filter report data

In [5]:
report_data = {}
for p_object in output:

    if p_object['workgroup_id'] in report_data:
        ...
    else:
        report_data["workgroup_id"] = p_object
    
